In [ ]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 불러오기
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  openai_api_key=openai_api_key,
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict(prompt);

In [ ]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}."),
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between Mexico and Thailand. Also, what is your name?"),
])

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand",
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
p = CommaOutputParser()
p.parse("Hello, how, are, you")

template = ChatPromptTemplate.from_messages([
  ("system", "You are a lit generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. DO NOT reply with anything else."),
  ("human", "{question}"),
])

prompt = template.format_messages(
  max_items=10,
  question="What are the planets"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({
  "max_items": 5,
  "question": "What are the Pokemons?"
})

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easys to find ingredients"),
  ("human", "I want to ook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegitarian international specialized on making traditional recipies vegetarian. You find alternative ingredients and explain theie preparation. You don't radically modify the  recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
  "cuisine": "indian",
})

In [9]:
poetry_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a poet who specializes in writing poems about programming languages. When a user conveys a particular programming language, write a poem with a short poem of about 100 characters using the language's name so that the language can be entered at least once."),
  ("human", "{language}")
])

poetry_chain = poetry_prompt | chat

desc_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a literary expert who specializes in reading and explaining poetry. Please focus on the subjects used in the poem to interpret the poem and briefly explain it in around 100 characters."),
  ("human", "{poetry}")
])

desc_chain = desc_prompt | chat

final_chain = {"poetry": poetry_chain} | desc_chain

final_chain.invoke({
  "language": "python"
})

In the world of code, where serpents slither,
Python reigns, a language that's a dither.The poem uses serpents and Python to symbolize complexity and unpredictability in the world of coding.

AIMessageChunk(content='The poem uses serpents and Python to symbolize complexity and unpredictability in the world of coding.')